In [1]:
import pandas as pd
import extruct
import requests
from time import sleep
from w3lib.html import get_base_url
from urllib.parse import urlparse
from bs4 import BeautifulSoup

In [2]:
# load in main csv as df
# Only used for repeat scrapes, not on itial run

df = pd.read_csv('../data/main.csv')

In [3]:
# dictionary of all manufacturers and models copied from Carvana site

test_dic = {'Acura': ['ILX', 'MDX', 'MDX-Sport-Hybrid', 'RDX', 'RL', 'RLX', 'RLX-Sport-Hybrid', 'TL', 'TLX', 'TSX', 'ZDX'],
            'Alfa-Romeo': ['4C-Spider', 'Giulia', 'Stelvio'],
            'Audi': ['A3', 'A3-Sportback-e-tron', 'A4', 'A4-allroad', 'A5', 'A5 Sport', 'A6', 'A7', 'A8', 'allroad',
                     'e-tron', 'e-tron GT', 'e-tron-Sportback', 'Q3', 'Q5', 'Q7', 'Q8', 'RS 3', 'RS 5', 'RS 6', 'S3',
                     'S4', 'S5', 'S6', 'S7', 'S8', 'SQ5', 'SQ7', 'TT'],
            'BMW': ['1 Series', '2 Series', '3 Series', '4 Series', '5 Series', '6 Series', '7 Series', '8 Series',
                    'i3', 'i8', 'M2', 'M3', 'M4', 'M5', 'M6', 'X1', 'X2', 'X3', 'X3 M', 'X4', 'X4 M', 'X5', 'X5 M', 'X6', 'X6 M', 'X7', 'Z4'],
            'Buick': ['Cascada', 'Enclave', 'Encore', 'Encore GX', 'Envision', 'LaCrosse', 'Lucerne', 'Regal', 'Regal Sportback', 'Regal TourX', 'Verano'],
            'Cadillac': ['ATS', 'ATS-V', 'CT4', 'CT5', 'CT6', 'CTS', 'CTS-V', 'DTS', 'ELR', 'Escalade', 'Escalade ESV', 'Escalade EXT', 'SRX', 'STS', 'XT4', 'XT5', 'XT6', 'XTS'],
            'Chevrolet': ['Avalanche', 'Aveo', 'Blazer', 'Bolt EV', 'Camaro', 'Captiva Sport', 'City Express', 'Cobalt', 'Colorado Crew Cab',
                          'Colorado Extended Cab', 'Colorado Regular Cab', 'Corvette', 'Cruze', 'Cruze Limited', 'Equinox', 'Express 2500 Cargo', 'Express 2500 Passenger',
                          'Express 3500 Cargo', 'Express 3500 Passenger', 'HHR', 'Impala', 'Impala Limited', 'Malibu', 'Malibu Limited', 'Silverado 1500 Crew Cab',
                          'Silverado 1500 Double Cab', 'Silverado 1500 Extended Cab', 'Silverado 1500 Limited Crew Cab', 'Silverado 1500 Limited Double Cab',
                          'Silverado 1500 Limited Regular Cab', 'Silverado 1500 Regular Cab', 'Silverado 2500 HD Crew Cab', 'Silverado 2500 HD Double Cab',
                          'Silverado 2500 HD Regular Cab', 'Silverado 3500 HD Crew Cab', 'Silverado 3500 HD Double Cab', 'Silverado 3500 HD Regular Cab', 'Sonic', 'Spark',
                          'Spark EV', 'SS', 'Suburban', 'Suburban 1500', 'Suburban 2500', 'Tahoe', 'TrailBlazer', 'Traverse', 'Trax', 'Volt'],
            'Chrysler': ['200', '300', 'Pacifica', 'Pacifica Hybrid', 'PT Cruiser', 'Sebring', 'Town & Country', 'Voyager'],
            'Dodge': ['Avenger', 'Caliber', 'Challenger', 'Charger', 'Dart', 'Durango', 'Grand Caravan Passenger', 'Journey', 'Nitro', 'Ram 1500 Crew Cab', 'Ram 1500 Quad Cab',
                      'Ram 1500 Regular Cab'],
            'FIAT': ['124 Spider', '500', '500 Abarth', '500c', '500c Abarth', '500e', '500L', '500X'],
            'Ford': ['Bronco', 'Bronco Sport', 'C-MAX Energi', 'C-MAX Hybrid', 'Crown Victoria', 'E150 Cargo', 'E150 Passenger', 'E250 Cargo', 'E350 Super Duty Passenger',
                     'EcoSport', 'Edge', 'Escape', 'Escape Plug-in Hybrid', 'Expedition', 'Expedition EL', 'Expedition MAX', 'Explorer', 'Explorer Sport Trac', 'F150 Regular Cab',
                     'F150 Super Cab', 'F150 SuperCrew Cab', 'F250 Super Duty Crew Cab', 'F350 Super Duty Crew Cab', 'F450 Super Duty Crew Cab', 'Fiesta', 'Flex', 'Focus', 'Focus ST',
                     'Fusion', 'Fusion Energi', 'Maverick', 'Mustang', 'Mustang MACH-E', 'Ranger Regular Cab', 'Ranger Super Cab', 'Ranger SuperCab', 'Ranger SuperCrew', 'Taurus',
                     'Transit 150 Cargo Van', 'Transit 150 Van', 'Transit 150 Wagon', 'Transit 250 Cargo Van', 'Transit 250 Van', 'Transit 350 Passenger Van',
                     'Transit 350 Wagon', 'Transit Connect Cargo', 'Transit Connect Cargo Van', 'Transit Connect Passenger', 'Transit Connect Passenger Wagon'],
            'Genesis': ['G70', 'G80', 'G90', 'GV70', 'GV80'],
            'GMC': ['Acadia', 'Acadia Limited', 'Canyon Crew Cab', 'Canyon Extended Cab', 'Envoy', 'Savana 2500 Cargo', 'Savana 3500 Passenger', 'Sierra 1500 Crew Cab',
                    'Sierra 1500 Double Cab', 'Sierra 1500 Extended Cab', 'Sierra 1500 Limited Crew Cab', 'Sierra 1500 Limited Double Cab', 'Sierra 1500 Regular Cab',
                    'Sierra 2500 HD Crew Cab', 'Sierra 2500 HD Double Cab', 'Sierra 2500 HD Extended Cab', 'Sierra 2500 HD Regular Cab', 'Sierra 3500 HD Crew Cab',
                    'Sierra 3500 HD Double Cab', 'Sierra 3500 HD Regular Cab', 'Terrain', 'Yukon', 'Yukon XL', 'Yukon XL 1500'],
            'Honda': ['Accord', 'Accord Crosstour', 'Accord Hybrid', 'Civic', 'Civic Type R', 'Clarity Plug-in Hybrid', 'CR-V', 'CR-V Hybrid', 'CR-Z', 'Crosstour', 'Element', 'Fit',
                      'HR-V', 'Insight', 'Odyssey', 'Passport', 'Pilot', 'Ridgeline'],
            'HUMMER': ['H3', 'H3T'],
            'Hyundai': ['Accent', 'Azera', 'Elantra', 'Elantra GT', 'Elantra N', 'Equus', 'Genesis', 'Genesis Coupe', 'IONIQ 5', 'Ioniq Electric', 'Ioniq Hybrid',
                        'Ioniq Plug-in Hybrid', 'Kona', 'Kona Electric', 'Kona N', 'Palisade', 'Santa Cruz', 'Santa Fe', 'Santa Fe Hybrid', 'Santa Fe Sport', 'Santa Fe XL',
                        'Sonata', 'Sonata Hybrid', 'Sonata Plug-in Hybrid', 'Tucson', 'Tucson Hybrid', 'Veloster', 'Venue', 'Veracruz'],
            'INFINITI': ['EX', 'FX', 'G', 'JX', 'M', 'Q40', 'Q50', 'Q60', 'Q70', 'QX', 'QX30', 'QX50', 'QX55', 'QX60', 'QX70', 'QX80'],
            'Jaguar': ['E-PACE', 'F-PACE', 'F-TYPE', 'I-PACE', 'XE', 'XF', 'XJ', 'XK'],
            'Jeep': ['Cherokee', 'Compass', 'Gladiator', 'Grand Cherokee', 'Grand Cherokee L', 'Liberty', 'Patriot', 'Renegade', 'Wagoneer', 'Wrangler', 'Wrangler Unlimited',
                     'Wrangler Unlimited 4xe'],
            'Kia': ['Cadenza', 'Carnival', 'EV6', 'Forte', 'Forte Koup', 'Forte5', 'K5', 'K900', 'Niro', 'Niro EV', 'Niro Plug-in Hybrid', 'Optima', 'Optima Hybrid',
                    'Optima Plug-in Hybrid', 'Rio', 'Rondo', 'Sedona', 'Seltos', 'Sorento', 'Sorento Hybrid', 'Sorento Plug-in Hybrid', 'Soul', 'Soul EV', 'Sportage',
                    'Stinger', 'Telluride'],
            'Land-Rover': ['Defender 110', 'Discovery', 'Discovery Sport', 'LR2', 'LR4', 'Range Rover', 'Range Rover Evoque', 'Range Rover Sport', 'Range Rover Velar'],
            'Lexus': ['CT', 'ES', 'GS', 'GX', 'HS', 'IS', 'IS F', 'LC', 'LS', 'LX', 'NX', 'RC', 'RX', 'UX'],
            'Lincoln': ['Aviator', 'Continental', 'Corsair', 'MKC', 'MKS', 'MKT', 'MKX', 'MKZ', 'Nautilus', 'Navigator', 'Navigator L', 'Town Car'],
            'Maserati': ['Ghibli', 'GranTurismo', 'Levante', 'Quattroporte'],
            'Mazda': ['CX-3', 'CX-30', 'CX-5', 'CX-7', 'CX-9', 'MAZDA2', 'MAZDA3', 'MAZDA5', 'MAZDA6', 'MX-5 Miata', 'MX-5 Miata RF', 'Tribute'],
            'Mercedes-Benz': ['A-Class', 'B-Class', 'C-Class', 'CL-Class', 'CLA', 'CLA-Class', 'CLK-Class', 'CLS', 'CLS-Class', 'E-Class', 'G-Class', 'GL-Class', 'GLA',
                              'GLA-Class', 'GLB', 'GLC', 'GLC Coupe', 'GLE', 'GLE Coupe', 'GLK-Class', 'GLS', 'M-Class', 'Mercedes-AMG C-Class', 'Mercedes-AMG CLA',
                              'Mercedes-AMG E-Class', 'Mercedes-AMG GLA', 'Mercedes-AMG GLC', 'Mercedes-AMG GLC Coupe', 'Mercedes-AMG GLE', 'Mercedes-AMG GLE Coupe',
                              'Mercedes-AMG GT', 'Metris Cargo', 'Metris Passenger', 'Metris WORKER Passenger', 'R-Class', 'S-Class', 'SL', 'SL-Class', 'SLC', 'SLK',
                              'SLK-Class', 'Sprinter 2500 Cargo'],
            'Mercury': ['Grand Marquis', 'Mariner', 'Milan'],
            'MINI': ['Clubman', 'Convertible', 'Countryman', 'Coupe', 'Hardtop', 'Hardtop 2 Door', 'Hardtop 4 Door', 'Paceman', 'Roadster'],
            'Mitsubishi': ['Eclipse', 'Eclipse Cross', 'Endeavor', 'Galant', 'i-MiEV', 'Lancer', 'Lancer Evolution', 'Mirage', 'Mirage G4', 'Outlander', 'Outlander PHEV',
                           'Outlander Sport'],
            'Nissan': ['370Z', 'Altima', 'Armada', 'cube', 'Frontier Crew Cab', 'Frontier King Cab', 'GT-R', 'JUKE', 'Kicks', 'LEAF', 'Maxima', 'Murano', 'NV1500 Cargo',
                       'NV200', 'NV2500 HD Cargo', 'NV3500 HD Cargo', 'Pathfinder', 'Quest', 'Rogue', 'Rogue Select', 'Rogue Sport', 'Sentra', 'Titan Crew Cab',
                       'Titan King Cab', 'TITAN Single Cab', 'TITAN XD Crew Cab', 'TITAN XD King Cab', 'TITAN XD Single Cab', 'Versa', 'Versa Note', 'Xterra'],
            'Polestar': ['2'],
            'Pontiac': ['G5', 'G6', 'G8', 'Solstice', 'Vibe'],
            'Porsche': ['718 Boxster', '718 Cayman', '911', 'Boxster', 'Cayenne', 'Cayman', 'Macan', 'Panamera'],
            'Ram': ['1500 Classic Crew Cab', '1500 Classic Quad Cab', '1500 Classic Regular Cab', '1500 Crew Cab', '1500 Quad Cab', '1500 Regular Cab', '2500 Crew Cab',
                    '2500 Regular Cab', '3500 Crew Cab', 'C/V Tradesman', 'Dakota Crew Cab', 'ProMaster Cargo Van', 'ProMaster City', 'ProMaster Window Van'],
            'Saab': ['9-3', '9-5'],
            'Saturn': ['Aura', 'Outlook', 'SKY', 'VUE'],
            'Scion': ['FR-S', 'iA', 'iM', 'iQ', 'tC', 'xB', 'xD'],
            'smart': ['fortwo', 'fortwo cabrio', 'fortwo electric drive', 'fortwo EQ coupe'],
            'Subaru': ['Ascent', 'BRZ', 'Crosstrek', 'Forester', 'Impreza', 'Legacy', 'Outback', 'Tribeca', 'WRX', 'XV Crosstrek'],
            'Suzuki': ['Kizashi', 'SX4'],
            'Tesla': ['Model 3', 'Model S', 'Model X', 'Model Y'],
            'Toyota': ['4Runner', '86', 'Avalon', 'Avalon Hybrid', 'C-HR', 'Camry', 'Camry Hybrid', 'Corolla', 'Corolla Cross', 'Corolla Hatchback', 'Corolla Hybrid',
                       'Corolla iM', 'FJ Cruiser', 'GR Supra', 'GR86', 'Highlander', 'Highlander Hybrid', 'Land Cruiser', 'Matrix', 'Prius', 'Prius c', 'Prius Plug-in Hybrid',
                       'Prius Prime', 'Prius v', 'RAV4', 'RAV4 Hybrid', 'RAV4 Prime', 'Sequoia', 'Sienna', 'Tacoma Access Cab', 'Tacoma Double Cab', 'Tacoma Regular Cab',
                       'Tundra CrewMax', 'Tundra Double Cab', 'Tundra Regular Cab', 'Venza', 'Yaris', 'Yaris Hatchback', 'Yaris iA'],
            'Volkswagen': ['Arteon', 'Atlas', 'Atlas Cross Sport', 'Beetle', 'CC', 'e-Golf', 'Eos', 'Golf', 'Golf Alltrack', 'Golf GTI', 'Golf R', 'Golf SportWagen', 'GTI',
                           'ID.4', 'Jetta', 'Jetta GLI', 'Jetta SportWagen', 'New Beetle', 'Passat', 'Rabbit', 'Routan', 'Taos', 'Tiguan', 'Tiguan Limited', 'Touareg'],
            'Volvo': ['C30', 'C70', 'S40', 'S60', 'S80', 'S90', 'V60', 'V90', 'XC40', 'XC40 Recharge', 'XC60', 'XC70', 'XC90']
           }

In [4]:
# turn dictionary into useable urls:
# case has no effect, but some multi-word names require a '-' to work as a url

def url_maker(dic):
    url_list = []
    for key, values in dic.items():
        for value in values:
            edited = '-'.join(value.split())
            #for year in range(2012, 2022):
            url_list.append(f'https://www.carvana.com/cars/{key}-{edited}')
    return url_list

In [5]:
url_list = url_maker(test_dic)
#url_list

['https://www.carvana.com/cars/Acura-ILX',
 'https://www.carvana.com/cars/Acura-MDX',
 'https://www.carvana.com/cars/Acura-MDX-Sport-Hybrid',
 'https://www.carvana.com/cars/Acura-RDX',
 'https://www.carvana.com/cars/Acura-RL',
 'https://www.carvana.com/cars/Acura-RLX',
 'https://www.carvana.com/cars/Acura-RLX-Sport-Hybrid',
 'https://www.carvana.com/cars/Acura-TL',
 'https://www.carvana.com/cars/Acura-TLX',
 'https://www.carvana.com/cars/Acura-TSX',
 'https://www.carvana.com/cars/Acura-ZDX',
 'https://www.carvana.com/cars/Alfa-Romeo-4C-Spider',
 'https://www.carvana.com/cars/Alfa-Romeo-Giulia',
 'https://www.carvana.com/cars/Alfa-Romeo-Stelvio',
 'https://www.carvana.com/cars/Audi-A3',
 'https://www.carvana.com/cars/Audi-A3-Sportback-e-tron',
 'https://www.carvana.com/cars/Audi-A4',
 'https://www.carvana.com/cars/Audi-A4-allroad',
 'https://www.carvana.com/cars/Audi-A5',
 'https://www.carvana.com/cars/Audi-A5-Sport',
 'https://www.carvana.com/cars/Audi-A6',
 'https://www.carvana.com/ca

In [6]:
# most of the beginning 10 lines in this fuction to extract schema.org metadata (and some imports) taken from:
# https://practicaldatascience.co.uk/data-science/how-to-scrape-schemaorg-metadata-using-python

def extract_metadata(url_list):
    """Extract all metadata present in the page and return a dictionary of metadata lists. 
    
    Args:
        url (string): URL of page from which to extract metadata. 
    
    Returns: 
        metadata (dict): Dictionary of json-ld, microdata, and opengraph lists. 
        Each of the lists present within the dictionary contains multiple dictionaries.
    """
    df = None
    for url in url_list:
        sleep(15)
        headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                    'Accept-Encoding' : 'gzip, deflate',
                    'Accept-Language' : 'en-US,en;q=0.9',
                    'Connection' : 'keep-alive',
                    #'Referer': 'https://www.bing.com/',
                    'Upgrade-Insecure-Requests' : '1' ,
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.47'}
        r = requests.get(url, headers=headers)
        html = r.text
        base_url = get_base_url(r.text, r.url)
        metadata = extruct.extract(r.text, 
                                   base_url=base_url,
                                   uniform=True,
                                   syntaxes=['json-ld',
                                             'microdata',
                                             'opengraph'])
        soup = BeautifulSoup(html)
        # add a list of trims to add to the available schema.org data
        ulist = soup.find_all('div', {'class': 'trim-mileage'})
        trim_list = []
        for u in ulist:
            trim_list.append(u.span.text)
            
        # the follwing checks to make sure the program has not been rate-limited by checking for zeros
        # then adds the trims to the appropriate row if there are over ten cars of that model
        # and concatenates dataframes if this is not the first time run
        
        dic = metadata
        #print(len(trim_list), len(dic['json-ld']))
        if len(trim_list) == 0 and len(dic['json-ld']) == 0:
            print(f'appears blocked on: {url}')
            return df
        if len(trim_list) > 10 and len(trim_list) <= len(dic['json-ld']):
            if df is not None:
                temp = pd.DataFrame.from_dict(dic['json-ld'][:len(trim_list)])
                temp['trim'] = trim_list
                df = pd.concat([df, temp], ignore_index=True)
                print(url, len(trim_list))
            else:
                df = pd.DataFrame.from_dict(dic['json-ld'][:len(trim_list)])
                df['trim'] = trim_list
                print(url, len(trim_list))
    print('Done!')
    return df

In [7]:
# runs function over entire carvana site (but only 20 of each model max)
# and warns to make sure initial dataframe is not unintentionally overwritten

if df is not None:
    print('adding to previous df...')
    df = pd.concat([df, extract_metadata(url_list)], ignore_index=True)
else:
    print('WARNING: no previous df found!')
    df = extract_metadata(url_list)

adding to previous df...
https://www.carvana.com/cars/Acura-ILX 20
https://www.carvana.com/cars/Acura-MDX 20
https://www.carvana.com/cars/Acura-RDX 20
https://www.carvana.com/cars/Acura-RLX 13
https://www.carvana.com/cars/Acura-TL 20
https://www.carvana.com/cars/Acura-TLX 20
https://www.carvana.com/cars/Acura-TSX 20
https://www.carvana.com/cars/Alfa-Romeo-Giulia 20
https://www.carvana.com/cars/Alfa-Romeo-Stelvio 20
https://www.carvana.com/cars/Audi-A3 20
https://www.carvana.com/cars/Audi-A4 20
https://www.carvana.com/cars/Audi-A5 20
https://www.carvana.com/cars/Audi-A6 20
https://www.carvana.com/cars/Audi-A7 20
https://www.carvana.com/cars/Audi-A8 15
https://www.carvana.com/cars/Audi-allroad 11
https://www.carvana.com/cars/Audi-Q3 20
https://www.carvana.com/cars/Audi-Q5 20
https://www.carvana.com/cars/Audi-Q7 20
https://www.carvana.com/cars/Audi-S3 19
https://www.carvana.com/cars/Audi-S4 15
https://www.carvana.com/cars/Audi-S5 20
https://www.carvana.com/cars/Audi-SQ5 12
https://www.car

In [11]:
df

,@context,@type,itemCondition,name,modelDate,manufacturer,model,color,image,brand,description,mileageFromOdometer,sku,vehicleIdentificationNumber,offers,trim
0,http://schema.org,Vehicle,Used,2016 Acura ILX,2016,Acura,ILX,White,//cdnblob.fastly.carvana.io/2001879052/post-la...,Acura,Used 2016 Acura ILX undefined with 70774 miles...,70774,2001879052,19UDE2F38GA018851,"{'@type': 'Offer', 'price': '20990', 'priceCur...",base
1,http://schema.org,Vehicle,Used,2013 Acura ILX,2013,Acura,ILX,Silver,//cdnblob.fastly.carvana.io/2001896115/post-la...,Acura,Used 2013 Acura ILX undefined with 71460 miles...,71460,2001896115,19VDE1F71DE012340,"{'@type': 'Offer', 'price': '18990', 'priceCur...",2.0L
2,http://schema.org,Vehicle,Used,2017 Acura ILX,2017,Acura,ILX,Gray,//cdnblob.fastly.carvana.io/2001806803/post-la...,Acura,Used 2017 Acura ILX undefined with 59327 miles...,59327,2001806803,19UDE2F73HA004148,"{'@type': 'Offer', 'price': '22990', 'priceCur...",base
3,http://schema.org,Vehicle,Used,2016 Acura ILX,2016,Acura,ILX,Silver,//cdnblob.fastly.carvana.io/2001794936/post-la...,Acura,Used 2016 Acura ILX undefined with 42231 miles...,42231,2001794936,19UDE2F86GA004927,"{'@type': 'Offer', 'price': '23590', 'priceCur...",base
4,http://schema.org,Vehicle,Used,2015 Acura ILX,2015,Acura,ILX,White,//cdnblob.fastly.carvana.io/2001632816/post-la...,Acura,Used 2015 Acura ILX undefined with 96189 miles...,96189,2001632816,19VDE1F3XFE008674,"{'@type': 'Offer', 'price': '17990', 'priceCur...",base
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17777,http://schema.org,Vehicle,Used,2019 Volvo XC60,2019,Volvo,XC60,White,//cdnblob.fastly.carvana.io/2001790815/post-la...,Volvo,Used 2019 Volvo XC60 undefined with 40634 mile...,40634,2001790815,LYV102RL7KB196554,"{'@type': 'Offer', 'price': '39590', 'priceCur...",T5 Inscription
17778,http://schema.org,Vehicle,Used,2019 Volvo XC60,2019,Volvo,XC60,White,//cdnblob.fastly.carvana.io/2001376566/post-la...,Volvo,Used 2019 Volvo XC60 undefined with 29570 mile...,29570,2001376566,LYV102DK4KB231484,"{'@type': 'Offer', 'price': '36990', 'priceCur...",T5 Momentum
17791,http://schema.org,Vehicle,Used,2016 Volvo XC70,2016,Volvo,XC70,White,//cdnblob.fastly.carvana.io/2001944771/post-la...,Volvo,Used 2016 Volvo XC70 undefined with 63553 mile...,63553,2001944771,YV4612NM2G1257760,"{'@type': 'Offer', 'price': '29990', 'priceCur...",T5 Platinum
17807,http://schema.org,Vehicle,Used,2015 Volvo XC70,2015,Volvo,XC70,Brown,//cdnblob.fastly.carvana.io/2001916163/post-la...,Volvo,Used 2015 Volvo XC70 undefined with 59815 mile...,59815,2001916163,YV440MBK6F1216667,"{'@type': 'Offer', 'price': '24990', 'priceCur...",T5 Premier (2015.5)


In [12]:
# careful here!
# overwrite main dataframe

df.to_csv('../data/main.csv', index=False)

In [ ]:
#check for duplicates after a rerun

df.duplicated(subset='vehicleIdentificationNumber').sum()

In [10]:
# drop those duplicates to ensure unique cars by VIN in dataframe

df.drop_duplicates(subset='vehicleIdentificationNumber', inplace=True)

7142